# Implementation of the SPT Model - Version 2

Modifications: Johnny Esteves\ Author: Allen Pinjic - Created on June 9th, 2022

In [1]:
from astropy.io.fits import getdata
from astropy.table import Table
from astropy.cosmology import WMAP9 as cosmo

In [ ]:
import numpy as np
import pandas as pd
import arviz as az
import matplotlib.pyplot as plt
import pylab as plt
import pymc3 as pm
import aesara
import matplotlib.font_manager
import scipy.stats
import scipy.optimize
import math

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [ ]:
%matplotlib inline

In [ ]:
fname = '../data_set/sptecs_catalog_oct919.fits'

data = Table(getdata(fname))
data[:2]

In [ ]:
sz_signal = np.array(data['XI'])
lambda_chisq = np.array(data['LAMBDA_CHISQ'])
lambda_chisqE = np.array(data['LAMBDA_CHISQ_E'])
redshift = np.array(data['REDSHIFT'])

In [ ]:
# Also called lambda hat (as shown in the paper)
## Shows the measured cluster richness whose values are over 20

plt.hist(np.log10(data['M500'][lambda_chisq>20]*1e14))
# Why not display values that are over 40
# "Cross matching studies, we restrict ourselves to the joint DESY1 x SPT-SZ footprint and to 𝜆 >ˆ 40"
# "We match the 𝜆 > ˆ 40 redMaPPer sample with the SPT-SZ sample selected above SZE signal to noise 𝜉 > 4"

Define the Model

In [ ]:
from colossus.cosmology import cosmology
from colossus.lss import mass_function
cosmology.setCosmology('WMAP9')

def _halo_mass_function(M, z):
    return mass_function.massFunction(M, z, mdef = '500c', model = 'bocquet16')
halo_mass_function = np.vectorize(_halo_mass_function)

def E(z):
    # The Hubble constant at the value of z
    Hz = cosmo.H(z).value
    # The Hubble constant at z=0
    H0 = cosmo.H(0).value
    return (Hz/H0)

In [ ]:
# LogNormal models
# see https://en.wikipedia.org/wiki/Log-normal_distribution
M0 = 3e14
Ez0 = E(0)

#Insert the priors on the SZE scaling relation parameters that identify with SZE,
# along with a value for the mass (M) and redshift (z)
## References Equation 2
def ln_zeta_given_M(theta_sze,M,z):
    A_sze, B_sze, C_sze, scatter_sze = theta_sze
    return np.log(A_sze) + (B_sze)*np.log(M/M0) + (C_sze)*(np.log(E(z)/Ez0))

#Insert the priors on the SZE scaling relation parameters that identify with LAMBDA, 
# along with a value for the mass (M) and redshift (z)
# Identified with the 
## References Equation 3
def ln_lbd_given_M(theta_lambda,M,z):
    A_lambda, B_lambda, C_lambda, scatter_lambda = theta_lambda
    return np.log(A_lambda) + (B_lambda)*np.log(M/M0) + (C_lambda)*(np.log(E(z)/Ez0))

#Insert the value of the mean and standard deviation as the two parameters
# to find the log of the variance in a normal distribution
def logNormal_variance(mu,std):
    return (np.exp(std**2)-1)*np.exp(2*mu+std**2)
# the linear relation lnLbd and lnZeta are logNormal
# the scatter of a logNormal is different from a normal distribution

In [ ]:
# set up integration vectors
mvec = np.logspace(13.8, 15.5, 75)
lbdvec = np.linspace(3,1.2*np.max(lambda_chisq),150)
zetavec = np.linspace(1,1.1*np.max(sz_signal),75)
# lbdvec = np.exp(np.arange(np.log(5),np.log(1.2*np.max(lambda_chisq)),0.032))
# zetavec = np.exp(np.arange(np.log(1),np.log(1.1*np.max(sz_signal)),0.045))

print('Vector size')
print(mvec.size)
print(lbdvec.size)
print(zetavec.size)

In [ ]:
zvec = np.linspace(0., 1.3, 100)
zzv, mm = np.meshgrid(zvec, mvec)
from scipy import interpolate
halo_mass_function2 = interpolate.interp1d(zvec, halo_mass_function(mm, zzv), kind='cubic')

In [ ]:
# taking only points with a significant p_chisi/lbd_hat

def slice_array(y,alpha=1e-2):
    cy = np.cumsum(y/np.sum(y),axis=0)
    ilo,iup = np.interp([alpha,1-alpha],cy,np.arange(len(y))).astype(int)+(0,2)
    return ilo, iup

In [ ]:
from scipy.integrate import simps

# given: mvec, lbdvec and zetavec

mm, zz, ll = np.meshgrid(mvec, zetavec, lbdvec, indexing='ij')

def log_likelihood_vec2(theta, z, y, yerr, eps=1e-9):
    # defining variables
    lbd_hat, chisi = y[0], y[1]
    lbd_err = yerr
    probs = []
    for lbd_hat_i, lbd_err_i, chisi_i, z_i in zip(lbd_hat, lbd_err, chisi, z):
        probs.append(_log_likelihood2(theta, lbd_hat_i, lbd_err_i, chisi_i, z_i))    
    p = np.array(probs)/np.sum(probs)
    return np.sum(np.log(p))

In [ ]:
def _log_likelihood2(theta, lbd_hat_i, lbd_err_i, chisi_i, z_i):
    # unfolding theta
    A_lambda, B_lambda, C_lambda, scatter_lambda = theta[4:8]
    A_sze, B_sze, C_sze, scatter_sze = theta[:4]
    rho = theta[-1]
    
    # calling predictions;
    ln_lbd_pred = ln_lbd_given_M([A_lambda, B_lambda, C_lambda, scatter_lambda], mvec, z_i)
    ln_zeta_pred= ln_zeta_given_M([A_sze, B_sze, C_sze, scatter_sze], mvec, z_i)
    halo_mass_func = halo_mass_function2(z_i)
    
    # error probabilities
    p_chisi = prob_chisi(zetavec, chisi_i)
    p_lbd_hat = prob_lbd_hat(lbdvec, lbd_hat_i, lbd_err_i)
    
    # take only significant p_lbd_hat values
    llo, lup = slice_array(p_lbd_hat,alpha=1e-4)
    clo, cup = slice_array(p_chisi,alpha=1e-4)
    
    hmf = np.tile(halo_mass_func, (int(lup-llo), int(cup-clo), 1)).T
    ln_lbd_pred = np.tile(ln_lbd_pred, (int(lup-llo), int(cup-clo), 1)).T
    ln_zeta_pred = np.tile(ln_zeta_pred, (int(lup-llo), int(cup-clo), 1)).T
    
    # compute dn_dlbd_dzeta_integrand
    p_total_m = compute_dn_dlbd_dzeta_vec2(lbd_hat_i, lbd_err_i, chisi_i,
                                           scatter_lambda, scatter_sze, rho,
                                           ll[:,clo:cup,llo:lup],zz[:,clo:cup,llo:lup],
                                           ln_lbd_pred, ln_zeta_pred, hmf)
    # integrate over M
    p_lbd_zeta = np.trapz(p_total_m, x=mvec, axis=0)

    # integrate over zeta
    p_chisi = np.tile(p_chisi[clo:cup], (int(lup-llo), 1)).T
    p_lbd = np.trapz(p_lbd_zeta*p_chisi, x=zetavec[clo:cup], axis=0)

    # integrate over lambda
    p = np.trapz(p_lbd*p_lbd_hat[llo:lup], x=lbdvec[llo:lup], axis=0)
    return p#np.log(p)

In [ ]:
def compute_dn_dlbd_dzeta_vec2(lbd_hat_i, lbd_err_i, chisi_i, scatter_lambda, scatter_sze, rho,
                               lvec, zvec, ln_lbd_pred, ln_zeta_pred, hmf):
    # converting std to normal distribution
    s_zeta = logNormal_variance(ln_zeta_pred, scatter_sze)
    s_lambda = logNormal_variance(ln_lbd_pred, scatter_lambda)
    
    # avoid error messages
    rho_inv = (1-rho**2)
    rho_inv = np.where(rho_inv<=eps, -np.inf, 1/rho_inv)

    # defining standirized variables
    lbd_std = (lvec - np.exp(ln_lbd_pred))/s_lambda
    zeta_std = (zvec - np.exp(ln_zeta_pred))/s_zeta

    # lbd_likelihood
    lp_lbd  = -rho_inv*lbd_std**2/2

    # zeta likelihood
    lp_zeta = -rho_inv*zeta_std**2/2

    # corr likelihod
    lp_corr = rho*rho_inv*lbd_std*zeta_std

    lp_total_m = lp_lbd+lp_zeta+lp_corr
    p_total_m = np.exp(lp_total_m)*hmf
    return p_total_m


In [ ]:
# Implementing the log_likelihood_vec2 in the emcee code
# via writing to the prior functions.

# In reference to rho's (ρ) defintion it states:
# Parameters of the richness–mass relation defined
# in Eq. 11 (Bleem et al. 2019) 
## and the correlation coefficient, ρSZ−λ, between the SZ signal (ζ) and richness.

# ρ also defined as the correlation coefficient  
# that encodes the degree of correlation between the intrinsic scatters on the respective observables

In [ ]:
Np = 10
ix = np.where(lambda_chisq>30)[0][:Np] # take 10 points

z = redshift[ix]
chisi = sz_signal[ix]
lbd_hat = lambda_chisq[ix]
lbd_err = lambda_chisqE[ix]

In [ ]:
# for a given cluster, i.e. a vector (lbd_hat_i, chisi_i, z_i)

# ix = np.arange(len(lambda_chisq))[lambda_chisq>0][np.argmin(sz_signal[lambda_chisq>0])]
ix = np.arange(len(lambda_chisq))[lambda_chisq>0][np.argmax(lambda_chisq[lambda_chisq>0])]

lbd_hat_i = lambda_chisq[ix]
lbd_err_i = lambda_chisqE[ix]
chisi_i = sz_signal[ix]
z_i = redshift[ix]
print(lbd_hat_i)
print(chisi_i)

In [ ]:
# test function 
# debuging here
eps  =1e-9
lbd  = lbd_hat[0]
zeta = chisi[0]

theta = [5.24, 1.534, 0.465, 0.161, 76.9, 1.02, 0.29, 0.16, 0.8]

In [ ]:
from __future__ import print_function, division

import os
import sys
import numpy as np

# import emcee
import emcee

## HOW TO: Directly implement data

# MAIN QUESTIONS:
# 2. How to make logprior for both scatter_sze and scatter_lambda since
# they act like rho (then again, why would that make creating a logprior function for them not possible?)

### LAMBDA PRIORS ###

def logprior_A_lbd(theta):  
    lp = 0.
    
    # unpack the model parameters from the tuple
    # unfolding theta
    A_lambda, B_lambda, C_lambda, scatter_lambda = theta[4:8]
    A_sze, B_sze, C_sze, scatter_sze = theta[:4]
    rho = theta[-1]
    
    # uniform prior on c
    rhomin = 0. # lower range of prior (rho)
    rhomax = 1.  # upper range of prior (rho)
    
    # set prior to 1 (log prior to 0) if in the range and zero (-inf) outside the range 
    lp = 0. if ((rhomin < rho < rhomax) and (scatter_sze > 0) and (scatter_lambda > 0)) else -np.inf
    
    # Gaussian prior on A_lambda
    A_lambda_mu = __    # mean of the Gaussian prior
    A_lambda_sigma = __ # standard deviation of the Gaussian prior
    lp -= 0.5*((A_lambda - A_lambda_mu)/A_lambda_sigma)**2
    
    return lp

def logprior_B_lbd(theta):  
    lp = 0.
    
    # unpack the model parameters from the tuple
    # unfolding theta
    A_lambda, B_lambda, C_lambda, scatter_lambda = theta[4:8]
    A_sze, B_sze, C_sze, scatter_sze = theta[:4]
    rho = theta[-1]
    
    # uniform prior on c
    rhomin = 0. # lower range of prior (rho)
    rhomax = 1.  # upper range of prior (rho)
    
    # set prior to 1 (log prior to 0) if in the range and zero (-inf) outside the range 
    lp = 0. if ((rhomin < rho < rhomax) and (scatter_sze > 0) and (scatter_lambda > 0)) else -np.inf
    
    # Gaussian prior on A_lambda
    B_lambda_mu = __    # mean of the Gaussian prior
    B_lambda_sigma = __ # standard deviation of the Gaussian prior
    lp -= 0.5*((B_lambda - B_lambda_mu)/B_lambda_sigma)**2
    
    return lp

def logprior_C_lbd(theta):  
    lp = 0.
    
    # unpack the model parameters from the tuple
    # unfolding theta
    A_lambda, B_lambda, C_lambda, scatter_lambda = theta[4:8]
    A_sze, B_sze, C_sze, scatter_sze = theta[:4]
    rho = theta[-1]
    
    # uniform prior on c
    rhomin = 0. # lower range of prior (rho)
    rhomax = 1.  # upper range of prior (rho)
    
    # set prior to 1 (log prior to 0) if in the range and zero (-inf) outside the range 
    lp = 0. if ((rhomin < rho < rhomax) and (scatter_sze > 0) and (scatter_lambda > 0)) else -np.inf
    
    # Gaussian prior on A_lambda
    C_lambda_mu = __    # mean of the Gaussian prior
    C_lambda_sigma = __ # standard deviation of the Gaussian prior
    lp -= 0.5*((C_lambda - C_lambda_mu)/C_lambda_sigma)**2
    
    return lp

def logprior_scatter_lbd(theta):  
    lp = 0.
    
    # unpack the model parameters from the tuple
    # unfolding theta
    A_lambda, B_lambda, C_lambda, scatter_lambda = theta[4:8]
    A_sze, B_sze, C_sze, scatter_sze = theta[:4]
    rho = theta[-1]
    
    # uniform prior on c
    rhomin = 0. # lower range of prior (rho)
    rhomax = 1.  # upper range of prior (rho)
    
    # set prior to 1 (log prior to 0) if in the range and zero (-inf) outside the range 
    lp = 0. if ((rhomin < rho < rhomax) and (scatter_sze > 0) and (scatter_lambda > 0)) else -np.inf
    
    # Gaussian prior on A_lambda
    scatter_lambda_mu = __    # mean of the Gaussian prior
    scatter_lambda_sigma = __ # standard deviation of the Gaussian prior
    lp -= 0.5*((scatter_lambda - scatter_lambda_mu)/scatter_lambda_sigma)**2
    
    return lp

In [ ]:
### SZE PRIORS ###
def logprior_A_sze(theta):  
    lp = 0.
    
    # unpack the model parameters from the tuple
    # unfolding theta
    A_lambda, B_lambda, C_lambda, scatter_lambda = theta[4:8]
    A_sze, B_sze, C_sze, scatter_sze = theta[:4]
    rho = theta[-1]
    
    # uniform prior on c
    rhomin = 0. # lower range of prior (rho)
    rhomax = 1.  # upper range of prior (rho)
    
    # set prior to 1 (log prior to 0) if in the range and zero (-inf) outside the range 
    lp = 0. if ((rhomin < rho < rhomax) and (scatter_sze > 0) and (scatter_lambda > 0)) else -np.inf
    
    # Gaussian prior on A_lambda
    A_sze_mu = __    # mean of the Gaussian prior
    A_sze_sigma = __ # standard deviation of the Gaussian prior
    lp -= 0.5*((A_sze - A_sze_mu)/A_sze_sigma)**2

    
def logprior_B_sze(theta):  
    lp = 0.
    
    # unpack the model parameters from the tuple
    # unfolding theta
    A_lambda, B_lambda, C_lambda, scatter_lambda = theta[4:8]
    A_sze, B_sze, C_sze, scatter_sze = theta[:4]
    rho = theta[-1]
    
    # uniform prior on c
    rhomin = 0. # lower range of prior (rho)
    rhomax = 1.  # upper range of prior (rho)
    
    # set prior to 1 (log prior to 0) if in the range and zero (-inf) outside the range 
    lp = 0. if ((rhomin < rho < rhomax) and (scatter_sze > 0) and (scatter_lambda > 0)) else -np.inf
    
    # Gaussian prior on A_lambda
    B_sze_mu = __    # mean of the Gaussian prior
    B_sze_sigma = __ # standard deviation of the Gaussian prior
    lp -= 0.5*((B_sze - B_sze_mu)/B_sze_sigma)**2

def logprior_C_sze(theta):  
    lp = 0.
    
    # unpack the model parameters from the tuple
    # unfolding theta
    A_lambda, B_lambda, C_lambda, scatter_lambda = theta[4:8]
    A_sze, B_sze, C_sze, scatter_sze = theta[:4]
    rho = theta[-1]
    
    # uniform prior on c
    rhomin = 0. # lower range of prior (rho)
    rhomax = 1.  # upper range of prior (rho)
    
    # set prior to 1 (log prior to 0) if in the range and zero (-inf) outside the range 
    lp = 0. if ((rhomin < rho < rhomax) and (scatter_sze > 0) and (scatter_lambda > 0)) else -np.inf
    
    # Gaussian prior on A_lambda
    C_sze_mu = __    # mean of the Gaussian prior
    C_sze_sigma = __ # standard deviation of the Gaussian prior
    lp -= 0.5*((C_sze - C_sze_mu)/C_sze_sigma)**2

def logprior_scatter_sze(theta):  
    lp = 0.
    
    # unpack the model parameters from the tuple
    # unfolding theta
    A_lambda, B_lambda, C_lambda, scatter_lambda = theta[4:8]
    A_sze, B_sze, C_sze, scatter_sze = theta[:4]
    rho = theta[-1]
    
    # uniform prior on c
    rhomin = 0. # lower range of prior (rho)
    rhomax = 1.  # upper range of prior (rho)
    
    # set prior to 1 (log prior to 0) if in the range and zero (-inf) outside the range 
    lp = 0. if ((rhomin < rho < rhomax) and (scatter_sze > 0) and (scatter_lambda > 0)) else -np.inf
    
    # Gaussian prior on A_lambda
    scatter_sze_mu = __    # mean of the Gaussian prior
    scatter_sze_sigma = __ # standard deviation of the Gaussian prior
    lp -= 0.5*((scatter_sze - scatter_sze_mu)/scatter_sze_sigma)**2

In [ ]:
def loglikelihood(theta, lbd_hat_i, lbd_err_i, chisi_i, z_i):
    # Imported the parameters of Johnny's _log_likelihood2 since they
    # match the emcee template
    # 1. theta = theta
    # 2. data = lbd_hat_i
    # 3. sigma (standard deviation) = lbd_err_i
    # Why include chisi_i and z_i?
    """
    The natural logarithm of the joint likelihood.
    
    Args:
        theta (tuple): a sample containing individual parameter values
        data (list): the set of data/observations
        sigma (float): the standard deviation of the data points
        x (list): the abscissa values at which the data/model is defined
    
    Note:
        We do not include the normalisation constants (as discussed above).
    """
    
    # unpack the model parameters from the tuple
    A_lambda, B_lambda, C_lambda, scatter_lambda = theta[4:8]
    A_sze, B_sze, C_sze, scatter_sze = theta[:4]
    rho = theta[-1]
    
    # evaluate the model (assumes that the straight_line model is defined as above)
    md = straight_line(x, m, c)
    ### Which variables should I include in this statement above?
    
    # return the log likelihood
    return -0.5*np.sum(((md - lbd_hat_i)/(lbd_err_i))**2)

In [ ]:
def logposterior(theta, lbd_hat_i, lbd_err_i, chisi_i, z_i):
    """
    The natural logarithm of the joint posterior.
    
    Args:
        theta (tuple): a sample containing individual parameter values
        data (list): the set of data/observations
        sigma (float): the standard deviation of the data points
        x (list): the abscissa values at which the data/model is defined
    """
    
    lp = logprior(theta) # get the prior
    
    # if the prior is not finite return a probability of zero (log probability of -inf)
    if not np.isfinite(lp):
        return -np.inf
    
    # return the likeihood times the prior (log likelihood plus the log prior)
    return  lp + loglikelihood(theta, lbd_hat_i, lbd_err_i, chisi_i, z_i)

In [ ]:
Nens = 100   # number of ensemble points

rhomin = 0.  # lower range of prior
rhomax = 1.   # upper range of prior

mmu = 0.     # mean of the Gaussian prior
msigma = 10. # standard deviation of the Gaussian prior

mini = np.random.normal(mmu, msigma, Nens) # initial m points
cini = np.random.uniform(rhomin, rhomax, Nens) # initial c points

## How to create the initial points for our data?
## Initial points for each prior? 
## Should each prior have a '_ini' function? If so, then what should its parameters be?

In [ ]:
inisamples = np.array([mini, cini]).T # initial samples

ndims = inisamples.shape[1] # number of parameters/dimensions

Nburnin = 500   # number of burn-in samples
Nsamples = 500  # number of final posterior samples

# set additional args for the posterior (the data, the noise std. dev., and the abscissa)
argslist = (theta, lbd_hat_i, lbd_err_i, chisi_i, z_i)

In [ ]:
# set up the sampler
sampler = emcee.EnsembleSampler(Nens, ndims, logposterior, args=argslist)

# pass the initial samples and total number of samples required
sampler.run_mcmc(inisamples, Nsamples+Nburnin);

# extract the samples (removing the burn-in)
postsamples = sampler.chain[:, Nburnin:, :].reshape((-1, ndims))

In [ ]:
try:
    import matplotlib as mpl
    mpl.use("Agg") # force Matplotlib backend to Agg
    import corner # import corner.py
except ImportError:
    sys.exit(1)

print('Number of posterior samples is {}'.format(postsamples.shape[0]))

fig = corner.corner(postsamples, labels=[r"$m$", r"$c$"], truths=[m, c])
fig.savefig('emcee.png')